<a href="https://colab.research.google.com/github/amindell11/WildfireSpreadPrediction/blob/main/notebooks/Test_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up Git/SSH

In [1]:
# --- Mount Drive if you haven't ---
from google.colab import drive
drive.mount('/content/drive')

import os, subprocess, pathlib, re

# 1️⃣  Path to the private key you generated earlier
key_path = pathlib.Path("/content/drive/MyDrive/.ssh/id_ed25519_colab")
assert key_path.exists(), "❌  SSH key not found on Drive. Generate it first."

# 2️⃣  Start ssh‑agent and capture its env vars
agent_output = subprocess.check_output(["ssh-agent", "-s"], text=True)
sock = re.search(r"SSH_AUTH_SOCK=([^;]+);", agent_output).group(1)
pid  = re.search(r"SSH_AGENT_PID=([0-9]+);", agent_output).group(1)
os.environ["SSH_AUTH_SOCK"] = sock
os.environ["SSH_AGENT_PID"] = pid

# 3️⃣  Add the key to the agent
subprocess.run(["ssh-add", str(key_path)], check=True)

# 4️⃣  Ensure ~/.ssh exists and pre‑populate known_hosts for GitHub
ssh_dir = pathlib.Path.home() / ".ssh"
ssh_dir.mkdir(exist_ok=True)            # ← replaces `mkdir -p ~/.ssh`
known_hosts = ssh_dir / "known_hosts"
subprocess.run(
    ["ssh-keyscan", "-t", "rsa", "github.com"],
    check=True, text=True,
    stdout=open(known_hosts, "a")
)

print("✅ SSH agent ready — you can git push over SSH from this runtime.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SSH agent ready — you can git push over SSH from this runtime.


In [2]:
username = "amindell11"
repo = "WildfireSpreadPrediction"
repo_dir = f'/content/drive/MyDrive/repos/{repo}'

remote_url = f"git@github.com:{username}/{repo}.git"

%cd {repo_dir}
!git remote set-url origin {remote_url}
!git config --global user.email "amindell11@gmail.com"
!git config --global user.name "Arye Mindell"

/content/drive/MyDrive/repos/WildfireSpreadPrediction


In [4]:
!ssh -T git@github.com

Hi amindell11! You've successfully authenticated, but GitHub does not provide shell access.


In [ ]:
!git add -u                 # or  !git add .
!git status -s
commit_msg = input("Commit message: ").strip()
if not commit_msg:
    commit_msg = "colab update"
!git commit -m "{commit_msg}"
!git pull --rebase origin main
!git push origin main

M  notebooks/Test_Book.ipynb
M  wildfire/autoencoder.py
Commit message: added code to run basic autoencoder train/eval (doesn't work yet)
[main 6931a71] added code to run basic autoencoder train/eval (doesn't work yet)
 2 files changed, 6 insertions(+), 9 deletions(-)
 rewrite notebooks/Test_Book.ipynb (87%)
error: cannot pull with rebase: You have unstaged changes.
error: please commit or stash them.
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 3.23 KiB | 254.00 KiB/s, done.
Total 6 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To github.com:amindell11/WildfireSpreadPrediction.git
   afbf9f3..6931a71  main -> main


# Install Dataset and Program modules

In [3]:
#!pip install . --quiet
import sys
sys.path.append(repo_dir)

In [ ]:
#copy kaggle auth key from drive to local
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#download dataset to drive folder (Not necessary if dataset already downloaded)
!kaggle datasets download -d fantineh/next-day-wildfire-spread -p /content/drive/MyDrive/Kaggle/


Dataset URL: https://www.kaggle.com/datasets/fantineh/next-day-wildfire-spread
License(s): Attribution 4.0 International (CC BY 4.0)
next-day-wildfire-spread.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/drive/MyDrive/Kaggle/next-day-wildfire-spread.zip
replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord  y
y
y
y
y

replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_01.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_01.tfrecord  y
y
y
y
y
y

replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_test_00.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename:   infl

In [11]:
data_loc = '/content/drive/MyDrive/Kaggle/next-day-wildfire-spread'
cache_loc = '/content/data'
!rsync -a {data_loc}/*.tfrecord {cache_loc}/


file_pattern = cache_loc + '/next_day_wildfire_spread_train*'
file_pattern_test = cache_loc + '/next_day_wildfire_spread_test*'
file_pattern_evaluate = cache_loc + '/next_day_wildfire_spread_eval*'



In [10]:
!unzip /content/drive/MyDrive/Kaggle/next-day-wildfire-spread.zip -d /content/drive/MyDrive/Kaggle/next-day-wildfire-spread

Archive:  /content/drive/MyDrive/Kaggle/next-day-wildfire-spread.zip
replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord  
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_01.tfrecord  yA

  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_test_00.tfrecord  
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_test_01.tfrecord  
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_train_00.tfrecord  
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_train_01.tfrecord  
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_train_02.tfreco

In [12]:
import wildfire.autoencoder as auto_enc
import wildfire.utils as utils

config={
    "learning_rate": 0.0005,
    "architecture": "Convolutional Autoencoder",
    "dataset": "NDWS",
    "optimizer": "adam",
    "epochs": 2,
    "batch_size": 2,
    "dropout_rate": 0.10
}

kwargs = auto_enc.get_dataset_args(config.get('batch_size'))
evalargs = kwargs.copy()
evalargs['batch_size'] = 100
dataset = utils.get_dataset(file_pattern, **kwargs)
dataset_test = utils.get_dataset(file_pattern_test, **kwargs)
dataset_eval = utils.get_dataset(file_pattern_evaluate, **evalargs)

model = auto_enc.build_autoencoder(input_shape=(64, 64, 12), dropout_rate=config['dropout_rate'])
auto_enc.train_and_evaluate_model(model, dataset, dataset_test, dataset_eval,config)


test
Epoch 1/2
   5253/Unknown 402s 75ms/step - binary_io_u_3: 0.4962 - loss: 0.0551

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


5253/5253 ━━━━━━━━━━━━━━━━━━━━ 411s 77ms/step - binary_io_u_3: 0.4962 - loss: 0.0551 - val_binary_io_u_3: 0.4934 - val_loss: 0.0515
Epoch 2/2
5253/5253 ━━━━━━━━━━━━━━━━━━━━ 443s 77ms/step - binary_io_u_3: 0.5066 - loss: 0.0397 - val_binary_io_u_3: 0.4958 - val_loss: 0.0478
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 707ms/step - binary_io_u_3: 0.4937 - loss: 0.0536


[0.059904228895902634, 0.4927261173725128]